In [1]:
import numpy as np
import pandas as pd


from keras.models import Sequential
from keras.layers import Dense
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from math import sqrt

Using TensorFlow backend.


# Dane

In [2]:
data = pd.read_csv('dane-pre.csv')

data.head(14)

,timestamp,MpKrakAlKras-PM10-1g,Weather,Temp(C),Wind(m/s),Direction(deg),Humidity,Barometer(mbar),PM10_Forecast_3h,PM10_Forecast_8h,PM10_Forecast_12h
0,2017-01-01 01:00:00,213.733,Ice fog.,-6.0,1.0,0.0,0.93,10.24,244.690,187.320,121.805
1,2017-01-01 02:00:00,213.868,Ice fog.,-7.0,2.0,190.0,0.93,10.24,224.673,181.194,119.096
2,2017-01-01 03:00:00,221.923,Ice fog.,-4.0,3.0,200.0,0.80,10.23,212.159,161.198,121.681
3,2017-01-01 04:00:00,244.690,Ice fog.,-7.0,3.0,210.0,0.86,10.22,198.375,136.218,138.353
4,2017-01-01 05:00:00,224.673,Ice fog.,-6.0,3.0,200.0,0.80,10.22,191.881,121.805,150.415
5,2017-01-01 06:00:00,212.159,Ice fog.,-7.0,2.0,190.0,0.86,10.22,187.320,119.096,161.587
6,2017-01-01 07:00:00,198.375,Ice fog.,-8.0,3.0,210.0,0.86,10.21,181.194,121.681,197.133
7,2017-01-01 08:00:00,191.881,Ice fog.,-8.0,1.0,0.0,0.93,10.21,161.198,138.353,217.923
8,2017-01-01 09:00:00,187.320,Ice fog.,-6.0,2.0,200.0,0.86,10.21,136.218,150.415,250.248
9,2017-01-01 10:00:00,181.194,Ice fog.,-3.0,2.0,200.0,0.74,10.21,121.805,161.587,227.516


In [3]:
data.Weather.unique()
weather_dict = {}
weathers = data.Weather.unique()

for num, name in enumerate(weathers, start=1):
    weather_dict[name] = num
#weather_dict


In [5]:
#data["hour"] = data["timestamp"].apply(lambda x:x[-8:-6]) #pora dnia chyba ma znaczenie (godziny szczytu itd)
#data["month"] = data["timestamp"].apply(lambda x:x[5:7]) #pora roku też może mieć wpływ

X = data[["MpKrakAlKras-PM10-1g","Direction(deg)","Temp(C)","Wind(m/s)","Humidity","Barometer(mbar)"]]
#"hour","month",
y = data[["PM10_Forecast_3h","PM10_Forecast_8h","PM10_Forecast_12h"]]

In [6]:
scaler = StandardScaler()
X_scale = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X_scale

,MpKrakAlKras-PM10-1g,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar)
0,3.406057,-1.229537,-1.641839,-1.06963,0.922469,1.009585
1,3.408971,0.461359,-1.747499,-0.62705,0.922469,1.009585
2,3.582857,0.550354,-1.430521,-0.18447,0.165148,0.881591
3,4.074335,0.639348,-1.747499,-0.18447,0.514681,0.753597
4,3.642222,0.550354,-1.641839,-0.18447,0.165148,0.753597
...,...,...,...,...,...,...
17298,-1.043013,1.173316,-0.796564,0.25811,0.922469,1.009585
17299,-1.012067,1.351305,-0.796564,0.25811,0.572936,1.137579
17300,-0.966574,1.440300,-0.690905,0.70069,0.223403,1.265572
17301,-0.969935,1.440300,-0.690905,1.14327,0.223403,1.265572


In [7]:
#weather_id trzeba by dodać jako one-hot a nie słownik, tylko że wtedy mamy 100 dodatkowych kolumn

X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size=0.1,
                                                    random_state=0)

In [8]:
X_scale.head(10)
#y.head(10)

,MpKrakAlKras-PM10-1g,Direction(deg),Temp(C),Wind(m/s),Humidity,Barometer(mbar)
0,3.406057,-1.229537,-1.641839,-1.06963,0.922469,1.009585
1,3.408971,0.461359,-1.747499,-0.62705,0.922469,1.009585
2,3.582857,0.550354,-1.430521,-0.18447,0.165148,0.881591
3,4.074335,0.639348,-1.747499,-0.18447,0.514681,0.753597
4,3.642222,0.550354,-1.641839,-0.18447,0.165148,0.753597
5,3.372078,0.461359,-1.747499,-0.62705,0.514681,0.753597
6,3.074519,0.639348,-1.853158,-0.18447,0.514681,0.625604
7,2.934331,-1.229537,-1.853158,-1.06963,0.922469,0.625604
8,2.835871,0.550354,-1.641839,-0.62705,0.514681,0.625604
9,2.703627,0.550354,-1.324861,-0.62705,-0.184385,0.625604


# Model

In [9]:
model = Sequential()
model.add(Dense(12, input_dim=6, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(3, activation='linear'))

In [10]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=20, verbose = 1)

Epoch 1/10
15572/15572 [==============================] - 5s 316us/step - loss: 4885.5141 - mse: 4885.5146 - mae: 52.6353
Epoch 2/10
15572/15572 [==============================] - 2s 114us/step - loss: 4268.7816 - mse: 4268.7793 - mae: 46.4864
Epoch 3/10
15572/15572 [==============================] - 2s 106us/step - loss: 3703.1700 - mse: 3703.1714 - mae: 40.4355
Epoch 4/10
15572/15572 [==============================] - 2s 120us/step - loss: 3293.0904 - mse: 3293.0918 - mae: 36.0956
Epoch 5/10
15572/15572 [==============================] - 2s 128us/step - loss: 2944.7251 - mse: 2944.7239 - mae: 32.7583
Epoch 6/10
15572/15572 [==============================] - 2s 130us/step - loss: 2623.6354 - mse: 2623.6367 - mae: 30.1773
Epoch 7/10
15572/15572 [==============================] - 2s 115us/step - loss: 2393.2653 - mse: 2393.2649 - mae: 28.3144
Epoch 8/10
15572/15572 [==============================] - 3s 162us/step - loss: 2205.7262 - mse: 2205.7261 - mae: 27.0059
Epoch 9/10
15572/15572 [

In [12]:
y_pred = model.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
rms

45.52867273362486

In [22]:
now = np.array([[20,210,-8,3,0.8 ,10]])
nnn=scaler.transform(now)
pred = model.predict(nnn)
pred


array([[47.432163, 47.463924, 47.095634]], dtype=float32)